In [1]:
from qiskit.transpiler import PassManager, InstructionDurations

In [2]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [3]:
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_guadalupe')

In [4]:
from src.tools.DD_insertion import construct_bv_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis

In [5]:
from qiskit.circuit.library import QFT, GraphState

In [6]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 15):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 15):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [7]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [8]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 15):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [10]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)


In [10]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
if applied DD for None: False
if applied DD for None: False
if applied DD for None: False
if applied DD for None: False
629505231ea57f25eaebd46c
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
629505259baeab53b24acb95
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295052d0212ff4423251d64
if applied DD for None: False
if applied DD 

In [11]:
for job_id in bv_job_ids:
    print(job_id)

629505231ea57f25eaebd46c
629505259baeab53b24acb95
6295052d0212ff4423251d64
629505304ad655d5639e48c1
6295053332e176cd7093c80c
629505351ea57fa491ebd46d
6295053816cdc4ea7ef25c62
6295053b3bb4af56d3b9532b
6295053dbc0d02bb44dbeb17
629505419baeab4ce04acb97
6295054332e176538193c80d
62950546bc0d021401dbeb18


In [9]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '629505231ea57f25eaebd46c',
    '629505259baeab53b24acb95',
    '6295052d0212ff4423251d64',
    '629505304ad655d5639e48c1',
    '6295053332e176cd7093c80c',
    '629505351ea57fa491ebd46d',
    '6295053816cdc4ea7ef25c62',
    '6295053b3bb4af56d3b9532b',
    '6295053dbc0d02bb44dbeb17',
    '629505419baeab4ce04acb97',
    '6295054332e176538193c80d',
    '62950546bc0d021401dbeb18',

]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        obtain_results = result_count.get(ideal_result, 0)
        pst.append(obtain_results/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', 'BV13', 'BV14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.4705810546875
pst result of hahn_X: 0.478271484375
pst result of hahn_Y: 0.475830078125
pst result of CP: 0.475341796875
pst result of CPMG: 0.46435546875
pst result of xy4: 0.4627685546875
pst result of xy8: 0.46875
pst result of xy16: 0.46142578125
pst result of udd1: 0.469482421875
pst result of udd2: 0.4730224609375
pst result of kdd: 0.4677734375
---------Result of circuit BV4 -------------
pst result of no_dd: 0.446533203125
pst result of hahn_X: 0.451904296875
pst result of hahn_Y: 0.45361328125
pst result of CP: 0.457275390625
pst result of CPMG: 0.4571533203125
pst result of xy4: 0.4542236328125
pst result of xy8: 0.4566650390625
pst result of xy16: 0.4508056640625
pst result of udd1: 0.4552001953125
pst result of udd2: 0.45166015625
pst result of kdd: 0.4447021484375
---------Result of circuit BV5 -------------
pst result of no_dd: 0.16162109375
pst result of hahn_X: 0.1595458984375
pst result of hahn_Y: 0.16

In [11]:
# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []
# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i/result_data[0] for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = list(data[1:, 6])
# xy16.insert(0, None)
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = list(data[1:, 9])
# kdd.insert(0, None)

# circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12',]

# x_ticks = circuit_names[:7]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2][:7], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2][:7], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2][:7], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2][:7], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2][:7], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2][:7], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2][:7], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2][:7], linestyle='--', marker='*', color='m', label='kdd')

# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_guadalupe_0530.pdf', bbox_inches='tight', pad_inches=0)

In [12]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295054832e176369493c80e
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295054a9baeab00ca4acb98
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295054b9baeab5c144acb99
if applied DD for None: False
if applied DD for N

In [13]:
for job_id in gs_job_ids:
    print(job_id)

6295054832e176369493c80e
6295054a9baeab00ca4acb98
6295054b9baeab5c144acb99
6295054d1ea57fc341ebd46e
6295054f0212ff78e7251d65
62950551bc0d0281f0dbeb19
6295055232e176fec693c80f
6295055432e176319993c810
6295055632e17626f493c811
629505589baeab84894acb9a
6295055a3bb4af0e74b9532d
6295055d16cdc4e197f25c63


In [12]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '6295054832e176369493c80e',
    '6295054a9baeab00ca4acb98',
    '6295054b9baeab5c144acb99',
    '6295054d1ea57fc341ebd46e',
    '6295054f0212ff78e7251d65',
    '62950551bc0d0281f0dbeb19',
    '6295055232e176fec693c80f',
    '6295055432e176319993c810',
    '6295055632e17626f493c811',
    '629505589baeab84894acb9a',
    '6295055a3bb4af0e74b9532d',
    '6295055d16cdc4e197f25c63',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6', 'GS7', 'GS8', 'GS9', 'GS10', 'GS11', 'GS12', 'GS13', 'GS14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.03557936504052403
jsd result of hahn_X: 0.035610186602872095
jsd result of hahn_Y: 0.03520636425974248
jsd result of CP: 0.030306497346689953
jsd result of CPMG: 0.03301933669192905
jsd result of xy4: 0.03278737893730726
jsd result of xy8: 0.03431204689112896
jsd result of xy16: 0.04220427646702819
jsd result of udd1: 0.038921298859606226
jsd result of udd2: 0.056006126286375336
jsd result of kdd: 0.03715156228910083
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.03096743177080014
jsd result of hahn_X: 0.030392226207591917
jsd result of hahn_Y: 0.03670799250881035
jsd result of CP: 0.031343886948879775
jsd result of CPMG: 0.04570236593586195
jsd result of xy4: 0.047331330922588295
jsd result of xy8: 0.03345188223782429
jsd result of xy16: 0.03830596438302473
jsd result of udd1: 0.039805194351370615
jsd result of udd2: 0.058328307614150234
jsd result of kdd: 0.045411349667854754
---------Result of c

In [14]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_guadalupe_T_0530.pdf', bbox_inches='tight', pad_inches=0)

In [14]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295056e1ea57f5c85ebd46f
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295057116cdc4c2d0f25c64
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
629505733bb4af2887b9532e
if applied DD for None: False
if applied DD for N

In [15]:
for job_id in qft_job_ids:
    print(job_id)

6295056e1ea57f5c85ebd46f
6295057116cdc4c2d0f25c64
629505733bb4af2887b9532e
62950576bc0d02fcd7dbeb1a
6295057a3bb4af30cab9532f
6295057f16cdc485baf25c65
62950584f33cdbc7d52e04e3
6295058b16cdc424c1f25c67
6295059316cdc46f1cf25c68
6295059c4bb975b539efb2dd
629505a516cdc42905f25c69
629505b016cdc49a3df25c6a


In [15]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '6295056e1ea57f5c85ebd46f',
    '6295057116cdc4c2d0f25c64',
    '629505733bb4af2887b9532e',
    '62950576bc0d02fcd7dbeb1a',
    '6295057a3bb4af30cab9532f',
    '6295057f16cdc485baf25c65',
    '62950584f33cdbc7d52e04e3',
    '6295058b16cdc424c1f25c67',
    '6295059316cdc46f1cf25c68',
    '6295059c4bb975b539efb2dd',
    '629505a516cdc42905f25c69',
    '629505b016cdc49a3df25c6a'
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', 'QFT7', 'QFT8', 'QFT9', 'QFT10', 'QFT11', 'QFT12', 'QFT13', 'QFT14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.028623867840334404
jsd result of hahn_X: 0.024024629670797434
jsd result of hahn_Y: 0.03323166785102849
jsd result of CP: 0.0309133693931966
jsd result of CPMG: 0.0533343329089908
jsd result of xy4: 0.030781644514410685
jsd result of xy8: 0.04456516367582531
jsd result of xy16: 0.052874023591637065
jsd result of udd1: 0.04011090519841904
jsd result of udd2: 0.1458339310613055
jsd result of kdd: 0.06140814839430951
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.06339387550268208
jsd result of hahn_X: 0.0682054823905667
jsd result of hahn_Y: 0.05863464045661833
jsd result of CP: 0.06662153025085717
jsd result of CPMG: 0.06841397820989394
jsd result of xy4: 0.08663785886277889
jsd result of xy8: 0.06897956456731745
jsd result of xy16: 0.07515157579003919
jsd result of udd1: 0.09113175198915574
jsd result of udd2: 0.06465319435191266
jsd result of kdd: 0.07997458418906678
---------Result of circuit Q

In [16]:
qft_results_data

{'QFT3': [0.028623867840334404,
  0.024024629670797434,
  0.03323166785102849,
  0.0309133693931966,
  0.0533343329089908,
  0.030781644514410685,
  0.04456516367582531,
  0.052874023591637065,
  0.04011090519841904,
  0.1458339310613055,
  0.06140814839430951],
 'QFT4': [0.06339387550268208,
  0.0682054823905667,
  0.05863464045661833,
  0.06662153025085717,
  0.06841397820989394,
  0.08663785886277889,
  0.06897956456731745,
  0.07515157579003919,
  0.09113175198915574,
  0.06465319435191266,
  0.07997458418906678],
 'QFT5': [0.07323255833795854,
  0.0766298889310113,
  0.07919114826286305,
  0.06500987522495119,
  0.07581241198374278,
  0.07371971330221067,
  0.060805591384062885,
  0.06966743350189472,
  0.08178450445736395,
  0.0996521404091094,
  0.06236482666922533],
 'QFT6': [0.0876124749921601,
  0.08043097623363563,
  0.08712089433198451,
  0.10538852825161428,
  0.09850747088643683,
  0.10214992148485309,
  0.0972593750902471,
  0.07635976470697585,
  0.1340428291384645,
  0

In [17]:
# import matplotlib.pyplot as plt
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_guadalupe_0530.pdf', bbox_inches='tight', pad_inches=0)